In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as T
import numpy as np
import pandas as pd
import matplotlib as plt
import os

from torchvision.models import resnet101

# os.chdir('')
from TrainTestLoops import train_with_wandb
from utils import train_val_test_split, save_results, set_random_seed
# os.chdir('')

In [2]:
set_random_seed(0)
labels = pd.read_csv("labels.csv")

kTrainSize = 100000
kTestSize = 10000
kValSize = 50000
kBatchSize = 128

train_transform = T.Compose(
        [T.ToTensor(),
        T.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
val_test_transform = T.Compose(
        [T.ToTensor(),
        T.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

train_loader, val_loader, test_loader = train_val_test_split(kTrainSize, kValSize, kTestSize,
                                                                train_transform, val_test_transform, kBatchSize, labels)

In [3]:
# upper cell first!!!
n_epochs = 1
n_classes = 200
model = resnet101(num_classes=n_classes).to(device)
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9)
scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[30, 40], gamma=0.1)
train_with_wandb(model, optimizer, n_epochs, train_loader, val_loader, model_name='resnet101',  scheduler=scheduler)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: bspanfilov (tmboris). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/boris/.netrc
wandb: Currently logged in as: bspanfilov. Use `wandb login --relogin` to force relogin


  0%|          | 0/391 [00:00<?, ?it/s]

In [ ]:
save_results(model, test_loader, kTestSize, filename='test_labels')